## **1. Importation des bibliothèques**

In [ ]:
!nvidia-smi

In [10]:
!pip install scikit-learn


[notice] A new release of pip available: 22.1.2 -> 24.3.1
[notice] To update, run: C:\Users\pat\.conda\envs\test_env\python.exe -m pip install --upgrade pip


In [2]:
!pip install tensorflow[and-cuda]

     -------------------------------------- 390.0/390.0 MB 6.9 MB/s eta 0:00:00
     -------------------------------------- 214.7/214.7 MB 7.1 MB/s eta 0:00:00
     --------------------------------------- 38.7/38.7 MB 11.7 MB/s eta 0:00:00
     -------------------------------------- 574.8/574.8 MB 4.1 MB/s eta 0:00:00
     --------------------------------------- 16.7/16.7 MB 12.8 MB/s eta 0:00:00
     --------------------------------------- 10.0/10.0 MB 13.0 MB/s eta 0:00:00
     ---------------------------------------- 889.9/889.9 kB ? eta 0:00:00
     -------------------------------------- 191.5/191.5 MB 6.0 MB/s eta 0:00:00
     --------------------------------------- 55.8/55.8 MB 10.4 MB/s eta 0:00:00
     -------------------------------------- 382.2/382.2 MB 7.7 MB/s eta 0:00:00
     -------------------------------------- 719.4/719.4 MB 2.2 MB/s eta 0:00:00
     -------------------------------------- 385.0/385.0 MB 5.1 MB/s eta 0:00:00
     -------------------------------------- 3

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1 requires fsspec, which is not installed.
pydantic 2.9.2 requires typing-extensions>=4.6.1; python_version < "3.13", but you have typing-extensions 4.5.0 which is incompatible.
pydantic-core 2.23.4 requires typing-extensions!=4.7.0,>=4.6.0, but you have typing-extensions 4.5.0 which is incompatible.
fastapi 0.115.3 requires typing-extensions>=4.8.0, but you have typing-extensions 4.5.0 which is incompatible.
torch 2.5.1 requires sympy==1.13.1, but you have sympy 1.13.3 which is incompatible.
torch 2.5.1 requires typing-extensions>=4.8.0, but you have typing-extensions 4.5.0 which is incompatible.
ipython 8.30.0 requires typing_extensions>=4.6; python_version < "3.12", but you have typing-extensions 4.5.0 which is incompatible.


In [20]:
!pip install --upgrade tensorflow


  Using cached tensorflow-2.18.0-cp310-cp310-win_amd64.whl (7.5 kB)
  Using cached tensorflow_intel-2.18.0-cp310-cp310-win_amd64.whl (390.0 MB)
     ---------------------------------------- 1.2/1.2 MB 5.2 MB/s eta 0:00:00
     --------------------------------------- 15.9/15.9 MB 34.4 MB/s eta 0:00:00
     ---------------------------------------- 5.5/5.5 MB 59.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.0
    Uninstalling numpy-2.2.0:
      Successfully uninstalled numpy-2.2.0

[notice] A new release of pip available: 22.1.2 -> 24.3.1
[notice] To update, run: C:\Users\pat\.conda\envs\test_env\python.exe -m pip install --upgrade pip


ERROR: Could not install packages due to an OSError: [WinError 5] Accès refusé: 'C:\\Users\\pat\\AppData\\Local\\Temp\\pip-uninstall-1ceylgt2\\linalg\\_umath_linalg.cp310-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [ ]:
import tensorflow as tf

print("Version TensorFlow :", tf.__version__)
print("GPU disponible :", tf.config.list_physical_devices('GPU'))


In [ ]:
# import os
# import random
# import numpy as np
# import matplotlib.pyplot as plt
# import tensorflow as tf
# from tensorflow.keras import backend as K
# from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, concatenate, UpSampling2D, Dropout
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.models import Model
# from tensorflow.keras.preprocessing import image
# from tensorflow.keras.utils import Sequence
# from sklearn.model_selection import train_test_split
# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
# import cv2
# import tqdm
# from sklearn.metrics import accuracy_score, jaccard_score

## **2. Préparation des données avec Générateurs de Batches**
 - Dans cette cellule, nous chargeons les images RGB d'origine ainsi que les masques de segmentation à partir des dossiers train, val, et test qui sont déjà préparés. Les images sont redimensionnées à une taille plus petite (256x256) pour rendre l'entraînement plus efficace, puis normalisées.
  - Le jeu de données est composé de 2 dossiers:
● leftImg8bit : ce dossier contient les images RGB d’origines. Ce sont les données d’éntrée
● gtFine : ce dossier contient les masques de segmentation. Ce sont les données de sortie
- En ce qui concerne les données de sortie, il y a plusieurs types de fichiers, mais nous ne nous intéresserons qu’aux
fichiers dont le nom se termine par “_labelIds.png”. Ce sont les masques correspondant aux images
RGB.

In [15]:
import os
import numpy as np
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing import image

# Définitions des chemins de fichiers avec chemins relatifs
base_dir = os.path.join(os.getcwd(), 'data')
image_dir_train = os.path.join(base_dir, 'leftImg8bit/train')
mask_dir_train = os.path.join(base_dir, 'gtFine/train')
image_dir_val = os.path.join(base_dir, 'leftImg8bit/val')
mask_dir_val = os.path.join(base_dir, 'gtFine/val')
image_dir_test = os.path.join(base_dir, 'leftImg8bit/test')
mask_dir_test = os.path.join(base_dir, 'gtFine/test')

# Générateur de données pour charger les images par batch
# Générateur de données pour charger les images par batch
class DataGenerator(Sequence):
    def __init__(self, image_dir, mask_dir, batch_size=16, img_size=(256, 256)):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.batch_size = batch_size
        self.img_size = img_size
        self.image_list = []

        # Parcours récursif des dossiers pour les images
        for root, _, files in os.walk(image_dir):
            for file in files:
                if file.endswith(('.jpg', '.jpeg', '.png')):
                    self.image_list.append(os.path.join(root, file))

        # Parcours récursif des dossiers pour les masques (uniquement `_labelIds.png`)
        self.mask_list = []
        for root, _, files in os.walk(mask_dir):
            for file in files:
                if file.endswith('_labelIds.png'):
                    self.mask_list.append(os.path.join(root, file))

        # S'assurer que les listes sont triées pour que chaque image corresponde au bon masque
        self.image_list.sort()
        self.mask_list.sort()

        self.indexes = np.arange(len(self.image_list))

    def __len__(self):
        return len(self.image_list) // self.batch_size

    def __getitem__(self, index):
        batch_indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        images, masks = [], []

        for i in batch_indexes:
            img_path = self.image_list[i]
            mask_path = self.mask_list[i]
            img = image.load_img(img_path, target_size=self.img_size)
            img = image.img_to_array(img) / 255.0
            mask = image.load_img(mask_path, color_mode="grayscale", target_size=self.img_size)
            mask = image.img_to_array(mask) / 255.0

            images.append(img)
            masks.append(mask)

        return np.array(images), np.array(masks)


# Création des générateurs de données pour train, validation et test
batch_size = 16
train_gen = DataGenerator(image_dir_train, mask_dir_train, batch_size=batch_size)
val_gen = DataGenerator(image_dir_val, mask_dir_val, batch_size=batch_size)
test_gen = DataGenerator(image_dir_test, mask_dir_test, batch_size=batch_size)


In [16]:
print(f"Nombre d'images dans le dossier d'entraînement : {len(train_gen.image_list)}")
print(f"Nombre de masques dans le dossier d'entraînement : {len(train_gen.mask_list)}")


Nombre d'images dans le dossier d'entraînement : 2975
Nombre de masques dans le dossier d'entraînement : 2975


In [7]:
for images, masks in train_gen:
    print(images.shape, masks.shape)
    break


(2, 256, 256, 3) (2, 256, 256, 1)


## **3. Développement et entraînement du modèle U-Net:**

**Explication :** Cette cellule contient la définition et la construction du modèle U-Net. Le modèle est entraîné sur les données divisées avec 10 epochs. U-Net est bien adapté à la segmentation d'images. Des callbacks ont été ajoutés pour effectuer un early stopping et sauvegarder le meilleur modèle.


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Option 1: Placer la construction du modèle et l'entraînement sur le CPU
with tf.device('/CPU:0'):
    # Function to build the U-Net model
    def unet_model(input_size=(256, 256, 3)):
        inputs = Input(input_size)

        # Downsampling path
        c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
        p1 = MaxPooling2D((2, 2))(c1)

        c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
        p2 = MaxPooling2D((2, 2))(c2)

        # Bottom (bottleneck)
        c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)

        # Upsampling path
        u4 = UpSampling2D((2, 2))(c3)
        u4 = concatenate([u4, c2])

        u5 = UpSampling2D((2, 2))(u4)
        u5 = concatenate([u5, c1])

        c5 = Conv2D(64, (3, 3), activation='relu', padding='same')(u5)
        outputs = Conv2D(1, (1, 1), activation='sigmoid')(c5)

        model = Model(inputs=[inputs], outputs=[outputs])
        return model

    # Compile the model
    model = unet_model()
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

    # Early stopping and model checkpoint
    early_stopping = EarlyStopping(patience=5, restore_best_weights=True, monitor='val_loss')
    model_checkpoint = ModelCheckpoint(os.path.join('models', 'best_model.keras'), save_best_only=True, monitor='val_loss')

    # Training the model on CPU
    history = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=10,
        callbacks=[early_stopping, model_checkpoint]
    )


## **4.  Comparaison des Modèles U-Net, VGG16-UNET, et U-Net Mini**

**Explication :** Ici, nous avons ajouté un modèle U-Net Mini, qui est une version plus légère et simplifiée du U-Net. Le modèle VGG16-UNET a été mis à jour pour inclure une couche de data augmentation interne afin de mieux généraliser les performances.


In [ ]:
# Importation des bibliothèques nécessaires
import os
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input, RandomFlip, RandomRotation, RandomZoom, Conv2D, UpSampling2D, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Désactiver complètement les GPU pour exécuter uniquement sur le CPU
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# Développement du modèle VGG16-UNET
def vgg16_unet_model(input_size=(256, 256, 3)):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_size)
    
    # Data augmentation intégrée
    inputs = Input(input_size)
    x = RandomFlip("horizontal")(inputs)
    x = RandomRotation(0.2)(x)
    x = RandomZoom(0.2)(x)
    
    # Passage de l'entrée dans le modèle VGG16
    block1_conv2 = base_model.get_layer('block1_conv2').output
    block2_conv2 = base_model.get_layer('block2_conv2').output
    block3_conv3 = base_model.get_layer('block3_conv3').output
    vgg_outputs = [block1_conv2, block2_conv2, block3_conv3]
    
    # Définir un nouveau modèle avec les sorties intermédiaires désirées
    vgg16_encoder = Model(inputs=base_model.input, outputs=vgg_outputs)
    
    # Passer les images par l'encodeur
    c1, c2, c3 = vgg16_encoder(x)
    
    # Partie remontante du U-Net avec des couches de décodage
    u4 = UpSampling2D((2, 2))(c3)
    u4 = concatenate([u4, c2])
    c4 = Conv2D(128, (3, 3), activation='relu', padding='same')(u4)
    
    u5 = UpSampling2D((2, 2))(c4)
    u5 = concatenate([u5, c1])
    c5 = Conv2D(64, (3, 3), activation='relu', padding='same')(u5)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c5)
    model = Model(inputs=[inputs], outputs=[outputs])
    
    return model

# Initialisation et compilation du modèle
vgg16_model = vgg16_unet_model()
vgg16_model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Définition des callbacks
early_stopping = EarlyStopping(patience=5, restore_best_weights=True, monitor='val_loss')
vgg16_checkpoint = ModelCheckpoint(os.path.join('models', 'vgg16_best_model.keras'), save_best_only=True, monitor='val_loss')

# Entraînement du modèle VGG16-UNET uniquement sur le CPU
def train_vgg16_on_cpu():
    with tf.device('/CPU:0'):  # Spécifie l'utilisation explicite du CPU
        vgg16_model.fit(
            train_gen,
            validation_data=val_gen,
            epochs=10,
            callbacks=[early_stopping, vgg16_checkpoint]
        )

# Appel de la fonction d'entraînement
train_vgg16_on_cpu()


In [17]:
import os
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.optimizers import Adam

# Développement du modèle U-Net Mini
def unet_mini_model(input_size=(256, 256, 3)):
    inputs = Input(input_size)
    c1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(64, (3, 3), activation='relu', padding='same')(p1)
    p2 = MaxPooling2D((2, 2))(c2)

    # Partie à la descente
    c3 = Conv2D(128, (3, 3), activation='relu', padding='same')(p2)

    # Partie remontante
    u4 = UpSampling2D((2, 2))(c3)
    u4 = concatenate([u4, c2])
    c4 = Conv2D(64, (3, 3), activation='relu', padding='same')(u4)

    u5 = UpSampling2D((2, 2))(c4)
    u5 = concatenate([u5, c1])
    c5 = Conv2D(32, (3, 3), activation='relu', padding='same')(u5)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c5)
    model = Model(inputs=[inputs], outputs=[outputs])
    
    return model

unet_mini_model = unet_mini_model()
unet_mini_model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Entraînement du modèle U-Net Mini
def train_unet_mini_on_gpu():
    with tf.device('/GPU:0'):
        unet_mini_model.fit(train_gen, validation_data=val_gen, epochs=10, callbacks=[early_stopping, unet_mini_checkpoint])
        
unet_mini_model()        

## 5. Conclusion, Comparaison et Visualisation des Modèles

** Cette cellule compare les trois modèles entraînés (U-Net classique, VGG16-UNET, et U-Net Mini) sur les métriques telles que l'accuracy, l'IoU, et le Dice Coefficient. Des graphiques sont créés pour visualiser les performances de chaque modèle. 

In [8]:
# Vérification des générateurs
print(f"Nombre d'images dans test_gen : {len(test_gen)}")
print(f"Nombre d'images dans val_gen : {len(val_gen)}")

# Si le générateur a des listes d'images et de masques, vérifie leur longueur
if hasattr(test_gen, 'image_list') and hasattr(test_gen, 'mask_list'):
    print(f"Nombre d'images dans test_gen : {len(test_gen.image_list)}")
    print(f"Nombre de masques dans test_gen : {len(test_gen.mask_list)}")

if hasattr(val_gen, 'image_list') and hasattr(val_gen, 'mask_list'):
    print(f"Nombre d'images dans val_gen : {len(val_gen.image_list)}")
    print(f"Nombre de masques dans val_gen : {len(val_gen.mask_list)}")

Nombre d'images dans test_gen : 762
Nombre d'images dans val_gen : 250
Nombre d'images dans test_gen : 1525
Nombre de masques dans test_gen : 1153
Nombre d'images dans val_gen : 500
Nombre de masques dans val_gen : 500


In [ ]:
!pip show tensorflow keras


In [12]:
import numpy as np
from tensorflow.keras.models import load_model
import os

# Charger les modèles sauvegardés
unet_model_path = os.path.join('models', 'unet_best_model.keras')
vgg16_model_path = os.path.join('models', 'vgg16_best_model.keras')
unet_mini_model_path = os.path.join('models', 'unet_mini_best_model.keras')

# Vérifier que les fichiers existent
assert os.path.exists(unet_model_path), f"Le modèle U-Net n'a pas été trouvé à {unet_model_path}"
assert os.path.exists(vgg16_model_path), f"Le modèle VGG16-UNET n'a pas été trouvé à {vgg16_model_path}"
assert os.path.exists(unet_mini_model_path), f"Le modèle U-Net Mini n'a pas été trouvé à {unet_mini_model_path}"

# Charger les modèles
unet_model = load_model(unet_model_path)
vgg16_model = load_model(vgg16_model_path)
unet_mini_model = load_model(unet_mini_model_path)

# Comparaison des performances des modèles
def compare_models(models, val_gen, test_gen):
    metrics = []
    for model in models:
        # Nom du modèle
        model_name = model.name if hasattr(model, 'name') else 'Unnamed Model'

        # Évaluation sur l'ensemble de validation
        print(f"Évaluation sur l'ensemble de validation pour le modèle: {model_name}")
        loss, accuracy = model.evaluate(val_gen, verbose=0)

        # Prédictions sur l'ensemble de test
        print(f"Prédictions sur l'ensemble de test pour le modèle: {model_name}")
        y_pred = model.predict(test_gen, verbose=0)
        y_pred_binary = (y_pred > 0.5).astype(np.uint8)

        # Récupérer les vraies étiquettes de l'ensemble de test
        y_test = np.concatenate([y for _, y in test_gen], axis=0)

        # Calcul de l'IoU (Intersection over Union)
        intersection = np.logical_and(y_test, y_pred_binary).sum()
        union = np.logical_or(y_test, y_pred_binary).sum()
        iou = intersection / union if union > 0 else 0

        # Calcul du coefficient de Dice
        dice = (2 * intersection) / (y_test.sum() + y_pred_binary.sum()) if (y_test.sum() + y_pred_binary.sum()) > 0 else 0

        # Ajouter les métriques du modèle à la liste
        metrics.append({
            'model': model_name,
            'loss': loss,
            'accuracy': accuracy,
            'iou': iou,
            'dice': dice
        })

    return metrics

# Liste des modèles à comparer
models = [unet_model, vgg16_model, unet_mini_model]

# Comparer les modèles
metrics = compare_models(models, val_gen, test_gen)

# Afficher les résultats
print("\nRésumé des performances des modèles :")
for metric in metrics:
    print(f"Model: {metric['model']}")
    print(f"  Loss: {metric['loss']:.4f}")
    print(f"  Accuracy: {metric['accuracy']:.4f}")
    print(f"  IoU: {metric['iou']:.4f}")
    print(f"  Dice: {metric['dice']:.4f}")

TypeError: Could not deserialize class 'Functional' because its parent module keras.src.models.functional cannot be imported. Full object config: {'module': 'keras.src.models.functional', 'class_name': 'Functional', 'config': {'name': 'functional_1', 'trainable': True, 'layers': [{'module': 'keras.layers', 'class_name': 'InputLayer', 'config': {'batch_shape': [None, 256, 256, 3], 'dtype': 'float32', 'sparse': False, 'name': 'input_layer_1'}, 'registered_name': None, 'name': 'input_layer_1', 'inbound_nodes': []}, {'module': 'keras.layers', 'class_name': 'Conv2D', 'config': {'name': 'conv2d_6', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}, 'filters': 16, 'kernel_size': [3, 3], 'strides': [1, 1], 'padding': 'same', 'data_format': 'channels_last', 'dilation_rate': [1, 1], 'groups': 1, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'module': 'keras.initializers', 'class_name': 'GlorotUnif

In [ ]:
# Visualisation des résultats
import pandas as pd
metrics_df = pd.DataFrame(metrics)

plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.plot(metrics_df['model'], metrics_df['accuracy'], label='Accuracy', marker='o')
plt.plot(metrics_df['model'], metrics_df['iou'], label='IoU', marker='o')
plt.plot(metrics_df['model'], metrics_df['dice'], label='Dice Coefficient', marker='o')
plt.title('Comparaison des métriques entre modèles')
plt.xlabel('Modèles')
plt.ylabel('Score')
plt.legend()

plt.subplot(1, 2, 2)
plt.bar(metrics_df['model'], metrics_df['loss'], color='red')
plt.title('Loss des différents modèles')
plt.xlabel('Modèles')
plt.ylabel('Loss')
plt.show()

## Visualisation des exemples de résultats pour chaque modèle

In [ ]:
# Fonction de visualisation des prédictions

def visualize_predictions(models, test_gen, num_examples=3):
    for model in models:
        print(f"Visualisation des prédictions pour le modèle: {model.name}")
        predictions = model.predict(test_gen)
        X_test, y_test = next(iter(test_gen))
        for i in range(num_examples):
            plt.figure(figsize=(12, 4))
            plt.subplot(1, 3, 1)
            plt.imshow(X_test[i])
            plt.title("Image d'origine")
            
            plt.subplot(1, 3, 2)
            plt.imshow(y_test[i].squeeze(), cmap='gray')
            plt.title("Masque réel")
            
            plt.subplot(1, 3, 3)
            plt.imshow(predictions[i].squeeze(), cmap='gray')
            plt.title("Masque prédit")
            
            plt.show()

# Visualiser quelques exemples de segmentation
visualize_predictions(models, test_gen)